In [ ]:
# Importar las clases necesarias
from pyspark.sql import SparkSession, Row
import requests
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from googletrans import Translator
import psycopg2
from psycopg2 import sql


# Crear una instancia de SparkSession
spark = SparkSession.builder.getOrCreate()


# Configurar la API key de OpenWeatherMap
api_key = "cb3c7af6f8a3112d069b2cd42e3d2651"

# Definir las provincias de Argentina
provincias = ["Buenos Aires", "Córdoba", "Santa Fe", "Mendoza", "Tucumán", "Parana", "Salta", "Resistencia", "Corrientes", "Misiones", "Santiago del Estero", "San Juan", "San Salvador de Jujuy", "Viedma", "Formosa", "Neuquén", "Rawson", "San Luis", "Catamarca", "La Rioja, AR", "Santa Rosa, AR", "Rio Gallegos", "Ushuaia"]
# Crear una lista para almacenar los datos del clima de cada provincia
datos_clima = []


#  Crea el traductor
translator = Translator()





# Obtener el clima para cada provincia
for provincia in provincias:
    # Realizar la solicitud a la API de clima y obtener los datos en formato JSON
    url = f"https://api.openweathermap.org/data/2.5/weather?q={provincia}&appid={api_key}&units=metric"
    respuesta = requests.get(url)
    data = respuesta.json()
    
    # Extraer los datos necesarios de la respuesta JSON y crear un objeto Row
    dato_clima = Row(
        ID=len(datos_clima)+1,
        longitud=data["coord"]["lon"],
        latitud=data["coord"]["lat"],
        descripcion = translator.translate(data["weather"][0]["description"], src='en', dest='es').text,
        temperatura=float(data["main"]["temp"]),  # Convertir a decimal utilizando float()
        temperatura_max=float(data["main"]["temp_max"]),
        temperatura_min=float(data["main"]["temp_min"]),
        presion=float(data["main"]["pressure"]),
        humedad="{}%".format(data["main"]["humidity"]),
        velocidad_viento=float(data["wind"]["speed"]),
        nombre=data["name"],
        zona_horaria=data["timezone"],
        pais=data["sys"]["country"]
    )

    # Agregar el objeto Row a la lista datos_clima
    datos_clima.append(dato_clima)
    
    # Definir el esquema del DataFrame
esquema = StructType([
    StructField("ID", IntegerType(), nullable=False),
    StructField("longitud", DoubleType(), nullable=True),
    StructField("latitud", DoubleType(), nullable=True),
    StructField("descripcion", StringType(), nullable=True),
    StructField("temperatura", DoubleType(), nullable=True),
    StructField("temperatura_max", DoubleType(), nullable=True),
    StructField("temperatura_min", DoubleType(), nullable=True),
    StructField("presion", DoubleType(), nullable=True),
    StructField("humedad", StringType(), nullable=True),
    StructField("velocidad_viento", DoubleType(), nullable=True),
    StructField("nombre", StringType(), nullable=True),
    StructField("zona_horaria", StringType(), nullable=True),
    StructField("pais", StringType(), nullable=True)
])

# Crear el DataFrame utilizando el esquema
dataframe = spark.createDataFrame(datos_clima, schema=esquema)

# Mostrar el DataFrame por pantalla
dataframe.show(dataframe.count(), truncate=False)


# Datos de conexión a Amazon Redshift
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
port = 5439
database = "data-engineer-database"
user = "mau_giovanetti_coderhouse"
password = "5K6m1tR3h9"
schema = "mau_giovanetti_coderhouse"  # Nombre del esquema donde se creará la tabla

# Crear la conexión a Amazon Redshift
conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=database,
    user=user,
    password=password
)

# Crear el cursor
cursor = conn.cursor()

# Crear la tabla en Amazon Redshift
table_name = "clima"
create_table_query = """
    CREATE TABLE IF NOT EXISTS {}.{} (
        ID INT PRIMARY KEY,
        longitud DOUBLE PRECISION,
        latitud DOUBLE PRECISION,
        descripcion VARCHAR(255),
        temperatura DOUBLE PRECISION,
        temperatura_max DOUBLE PRECISION,
        temperatura_min DOUBLE PRECISION,
        presion DOUBLE PRECISION,
        humedad VARCHAR(255),
        velocidad_viento DOUBLE PRECISION,
        nombre VARCHAR(255),
        zona_horaria VARCHAR(255),
        pais VARCHAR(255)
    )
""".format(schema, table_name)

cursor.execute(create_table_query)
conn.commit()

# Insertar los datos en la tabla
for dato_clima in datos_clima:
    id_clima = dato_clima.ID

    # Verificar si los datos ya existen en la tabla
    select_query = sql.SQL("""
        SELECT COUNT(*) FROM {}.{}
        WHERE ID = %s
    """).format(sql.Identifier(schema), sql.Identifier(table_name))

    cursor.execute(select_query, (id_clima,))
    result = cursor.fetchone()

    if result[0] == 0:
        # Los datos no existen, realizar la inserción
        insert_query = sql.SQL("""
            INSERT INTO {}.{} (ID, longitud, latitud, descripcion, temperatura, temperatura_max, temperatura_min, presion, humedad, velocidad_viento, nombre, zona_horaria, pais)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """).format(sql.Identifier(schema), sql.Identifier(table_name))

        cursor.execute(insert_query, dato_clima)
        conn.commit()
    else:
        print("Los datos con ID {} ya existen en la tabla.".format(id_clima))

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

print("El proceso ha finalizado correctamente.")